# Chat Completions）
使用 Chat Completions API 实现对话任务。Chat Completions 以消息列表作为输入，并返回模型生成的消息作为输出。尽管聊天格式旨在使多轮对话变得简单，但它同样适用于没有任何对话的单轮任务。主要请求参数说明：
- **`model` （string，必填）**：要使用的模型ID。有关哪些模型适用于Chat API的详细信息
- **`messages` （array，必填）**：迄今为止描述对话的消息列表
    - **`role` （string，必填）**：发送此消息的角色。`system` 、`user` 或 `assistant` 之一（一般用 user 发送用户问题，system 发送给模型提示信息）
    - **`content` （string，必填）**：消息的内容
    - **`name` （string，选填）**：此消息的发送者姓名。可以包含 a-z、A-Z、0-9 和下划线，最大长度为 64 个字符
- **`stream` （boolean，选填，是否按流的方式发送内容）**：当它设置为 true 时，API 会以 SSE（ Server Side Event ）方式返回内容。SSE 本质上是一个长链接，会持续不断地输出内容直到完成响应。如果不是做实时聊天，默认false即可。
- **`max_tokens` （integer，选填）**：在聊天补全中生成的最大 **tokens** 数，输入token和生成的token的总长度受模型上下文长度的限制。
- **`temperature` （number，选填，默认是 1）**：采样温度，在 0和 2 之间。较高的值，如0.8会使输出更随机，而较低的值，如0.2会使其更加集中和确定性。通常建议修改这个（`temperature` ）或者 `top_p` ，但两者不能同时存在，二选一。

## 开启聊天模式
使用 `messages` 记录迄今为止对话的消息列表

In [13]:
from openai import OpenAI
client = OpenAI()

messages=[
    {
        "role": "user", 
        "content": "Hello!"
    }
]


data = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages = messages
)

print(data)

ChatCompletion(id='chatcmpl-96YCmpQzHWNpGUsFGIkiqLIR8Ptbu', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I help you today?', role='assistant', function_call=None, tool_calls=None))], created=1711348480, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_3bc1b5746c', usage=CompletionUsage(completion_tokens=9, prompt_tokens=9, total_tokens=18))


In [14]:
# 从返回的数据中获取生成的消息
new_message = data.choices[0].message
# 打印 new_message
print(new_message)

ChatCompletionMessage(content='Hello! How can I help you today?', role='assistant', function_call=None, tool_calls=None)


In [15]:
# 将消息追加到 messages 列表中
messages.append(new_message)
print(messages)

[{'role': 'user', 'content': 'Hello!'}, ChatCompletionMessage(content='Hello! How can I help you today?', role='assistant', function_call=None, tool_calls=None)]


In [16]:
type(new_message)

openai.types.chat.chat_completion_message.ChatCompletionMessage

In [17]:
new_message.role

'assistant'

In [18]:
new_message.content

'Hello! How can I help you today?'

In [19]:
messages.pop()

ChatCompletionMessage(content='Hello! How can I help you today?', role='assistant', function_call=None, tool_calls=None)

In [20]:
print(messages)

[{'role': 'user', 'content': 'Hello!'}]


#### Prompt: OpenAIObject -> Dict

```
打印 messages 列表后发现数据类型不对，messages 输出如下：

print(messages)

[{'role': 'user', 'content': 'Hello!'}, <OpenAIObject at 0x7f27582c13f0> JSON: {
  "content": "Hello! How can I assist you today?",
  "role": "assistant"
}]

将OpenAIObject 转换为一个如下数据类型格式：

    {
        "role": "user", 
        "content": "Hello!"
    }
```

In [21]:
new_message = data.choices[0].message
new_message_dict = {"role": new_message.role, "content": new_message.content}
type(new_message_dict)

dict

In [22]:
print(new_message_dict)

{'role': 'assistant', 'content': 'Hello! How can I help you today?'}


In [23]:
# 将消息追加到 messages 列表中
messages.append(new_message_dict)

print(messages)

[{'role': 'user', 'content': 'Hello!'}, {'role': 'assistant', 'content': 'Hello! How can I help you today?'}]


## 新一轮对话

In [24]:
new_chat = {
    "role": "user",
    "content": "1.讲一个程序员才听得懂的冷笑话；2.今天是几号？3.明天星期几？"
}

messages.append(new_chat)
from pprint import pprint

pprint(messages)

[{'content': 'Hello!', 'role': 'user'},
 {'content': 'Hello! How can I help you today?', 'role': 'assistant'},
 {'content': '1.讲一个程序员才听得懂的冷笑话；2.今天是几号？3.明天星期几？', 'role': 'user'}]


In [25]:
data = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages
)

new_message = data.choices[0].message
print(new_message)

ChatCompletionMessage(content='1. 为什么程序员喜欢黑色的衣服？\n   因为黑色背景下的代码更容易看清楚！\n\n2. 今天是几号？请问您的当前时区和具体时间是多少？\n\n3. 明天星期几？同样需要知道您的当前时区和具体时间是多少。', role='assistant', function_call=None, tool_calls=None)


In [26]:
# 打印 new_messages 内容
print(new_message.content)

1. 为什么程序员喜欢黑色的衣服？
   因为黑色背景下的代码更容易看清楚！

2. 今天是几号？请问您的当前时区和具体时间是多少？

3. 明天星期几？同样需要知道您的当前时区和具体时间是多少。


## 使用多 Role 聊天对话
目前`role`参数支持3类身份： `system`, `user` `assistant`:

In [27]:
import openai

# 构造聊天记录
messages=[
    {"role": "system", "content": "你是一个乐于助人的体育界专家。"},
    {"role": "user", "content": "2008年奥运会是在哪里举行的？"},
]

data = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages
)

message = data.choices[0].message.content
print(message)

2008年夏季奥林匹克运动会是在中国的北京举行的。


In [28]:
# 添加 GPT 返回结果到聊天记录
messages.append({"role": "assistant", "content": message})
messages

[{'role': 'system', 'content': '你是一个乐于助人的体育界专家。'},
 {'role': 'user', 'content': '2008年奥运会是在哪里举行的？'},
 {'role': 'assistant', 'content': '2008年夏季奥林匹克运动会是在中国的北京举行的。'}]

In [30]:
# 第二轮对话
messages.append({"role": "user", "content": "1.金牌最多的是哪个国家？2.奖牌最多的是哪个国家？"})

messages

[{'role': 'system', 'content': '你是一个乐于助人的体育界专家。'},
 {'role': 'user', 'content': '2008年奥运会是在哪里举行的？'},
 {'role': 'assistant', 'content': '2008年夏季奥林匹克运动会是在中国的北京举行的。'},
 {'role': 'user', 'content': '1.金牌最多的是哪个国家？2.奖牌最多的是哪个国家？'},
 {'role': 'user', 'content': '1.金牌最多的是哪个国家？2.奖牌最多的是哪个国家？'}]

In [31]:
data = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages
)

message = data.choices[0].message.content
print(message)

1. 2008年北京奥运会中，金牌最多的国家是中国，他们获得了51枚金牌。
2. 2008年北京奥运会中，总奖牌数最多的国家同样是中国，共获得100枚奖牌（51枚金牌、21枚银牌、28枚铜牌）。


In [32]:
data = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[{'role': 'user', 'content': '1.金牌最多的是哪个国家？2.奖牌最多的是哪个国家？'}]
)

data.choices[0].message.content

'1. 美国是历届夏季奥运会金牌最多的国家，总共获得了1090枚金牌。\n2. 美国是历届夏季奥运会奖牌最多的国家，总共获得了2684枚奖牌。'